In [2]:
import pandas as pd
from thunderpack import ThunderReader
from openpyxl import load_workbook

In [3]:
#get list of patients from excel file
#include patient ID and encounter ID in the list
patients = pd.read_excel('/media/cdac-c-15/Thunderpack/Dropbox/cardiac arrest project/cardiac_filtered_orginal .xlsx')
patients = patients[['Unnamed: 0', 'BDSPPatientID', 'SiteID', 'BDSPEncounterID']]
patients.head()


,Unnamed: 0,BDSPPatientID,SiteID,BDSPEncounterID
0,43477,111189188,S0001,538929570
1,88576,111189188,S0001,538929570
2,102055,111189188,S0001,538929570
3,194219,111189188,S0001,538929570
4,132878,111189188,S0001,538929570


In [4]:
uniqueIDs = patients['BDSPPatientID']
uniqueIDs = uniqueIDs.drop_duplicates()
print(uniqueIDs)

0        111189188
12       111200203
13       111204957
18       111205248
19       111222562
           ...    
12131    122423260
12133    122449422
12136    122474334
12139    122479673
12141    122519256
Name: BDSPPatientID, Length: 1650, dtype: int64


In [5]:
# read into demographics file to make a dataframe that contains information about all patients
reader = ThunderReader('/media/cdac-c-15/Thunderpack/Dropbox/zz_EHR_Thunderpacks/MGB/thunderpack_demographics_MGB')
dfs = []
for i in range(1, (len(list(reader.keys())))):
    df = reader[f'demographics_partition_{i}']
    df = df[df['BDSPPatientID'].isin(uniqueIDs)]
    dfs.append(df)
    del df
demographics = pd.concat(dfs, axis=0, ignore_index=True)     #puts data into rows and columns

del dfs     # clean up my data a little bit
del reader

demographics = demographics.drop_duplicates(subset='BDSPPatientID')
print(len(demographics))      #print the number of notes found with those ICD codes
demographics.head()      #print the first few lines of the filtered list of notes

1650


,BDSPPatientID,DateOfBirth,DateOfDeath,DateOfDeathMARegistryData,PatientRace,EthnicGroupDSC,MaritalStatusDSC,ReligionDSC,LanguageDSC,VeteranStatusDSC,...,PrimaryCauseOfDeathDSC,UNOSPrimaryCauseOfDeathTXT,FirstContributoryCauseOfDeathDSC,UNOSContributoryCauseOfDeath01TXT,SecondContributoryCauseOfDeathDSC,UNOSContributoryCauseOfDeath02TXT,EducationLevelDSC,GenderIdentityDSC,SexAssignedAtBirthDSC,BDSPLastModifiedDTS
0,117039206,1967-10-05,2021-11-20,2021-11-20,White,Not Hispanic,Single,Atheist,English,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - College,None,None,2022-11-29 10:00:51.2500000
1,119106063,1963-06-28,None,None,Black or African American,Unavailable,Life Partner,Not Affiliated,English,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Other,None,None,2022-07-17 17:57:14.1830000
2,114917293,1950-04-12,2019-07-28,2019-07-28,Black or African American,Not Hispanic,Married/Civil Union,Roman Catholic,English,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - High School,None,None,2022-11-29 13:01:12.0500000
3,114886961,2000-03-12,None,None,Other,Hispanic,Single,Not Affiliated,Spanish,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - High School,Male,Male,2022-07-17 17:57:14.1830000
4,116442969,2001-05-19,2021-03-01,2021-03-01,Other,Hispanic,Single,Roman Catholic,Spanish,"No, Never Served or Is Currently Active",...,None,None,None,None,None,NaN,Graduated - High School,None,None,2022-11-29 14:03:15.2366667


In [6]:
# merge the dfs to make final df contain missing data
mergedInformation = patients.merge(demographics[['BDSPPatientID', 'PatientRace', 'EthnicGroupDSC']], on='BDSPPatientID', how='left')
mergedInformation.head()

,Unnamed: 0,BDSPPatientID,SiteID,BDSPEncounterID,PatientRace,EthnicGroupDSC
0,43477,111189188,S0001,538929570,Black or African American,Not Hispanic
1,88576,111189188,S0001,538929570,Black or African American,Not Hispanic
2,102055,111189188,S0001,538929570,Black or African American,Not Hispanic
3,194219,111189188,S0001,538929570,Black or African American,Not Hispanic
4,132878,111189188,S0001,538929570,Black or African American,Not Hispanic


In [7]:
excel_df = pd.read_excel('/media/cdac-c-15/Thunderpack/Dropbox/cardiac arrest project/cardiac_filtered_orginal .xlsx', sheet_name='cardiac_filtered_new')

In [8]:
# check if the dfs are able to be overlaid
print(len(excel_df))
print(len(mergedInformation))
print(len(patients))

12146
12146
12146


In [9]:
#merge excel sheet with existing data
# Assuming 'PatientID' and 'EncounterID' are columns in both DataFrames
# and we want to update the 'Data_Column' in the Excel DataFrame
merged_df = excel_df.merge(mergedInformation[['Unnamed: 0', 'BDSPPatientID', 'BDSPEncounterID', 'PatientRace', 'EthnicGroupDSC']], on=['Unnamed: 0', 'BDSPPatientID', 'BDSPEncounterID'], how='left')

print(len(merged_df))
merged_df.head()

12146


,Unnamed: 0,group,BDSPPatientID,SiteID,BonoboIndex,sparcnetID,HashFolderName,HashFileName,BidsFolder,SessionID_new,...,jae,jeavons,sunflower,dravet,eses,cjd,angelman,jeavonssunflower,PatientRace,EthnicGroupDSC
0,43477,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,7,...,0,0,0,0,0,0,0,NaN,Black or African American,Not Hispanic
1,88576,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,7,...,0,0,0,0,0,0,0,NaN,Black or African American,Not Hispanic
2,102055,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,6,...,0,0,0,0,0,0,0,NaN,Black or African American,Not Hispanic
3,194219,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,6,...,0,0,0,0,0,0,0,NaN,Black or African American,Not Hispanic
4,132878,9,111189188,S0001,NaN,pat0005,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,a81e3504efe7bde263ce334822e05527ed3c6aaa2dc779...,sub-S0001111189188,5,...,0,0,0,0,0,0,0,NaN,Black or African American,Not Hispanic


In [19]:
# get the information that I need from the merged df

# Select the 'race' and 'ethnicity' rows from the DataFrame
selected_rows = merged_df[['PatientRace', 'EthnicGroupDSC']]

# Convert the selected rows into lists
race_data = selected_rows['PatientRace'].tolist()
ethnicity_data = selected_rows['EthnicGroupDSC'].tolist()

In [22]:
# Load the workbook and the specific sheet
path = '/media/cdac-c-15/Thunderpack/Dropbox/cardiac arrest project/cardiac_filtered_orginal .xlsx'
sheet_name='cardiac_filtered_new'
workbook = load_workbook(path)
sheet = workbook[sheet_name]
# 23 is race
# 24 is ethnicity
# starting row is 2

starting_row = 2

# Add new columns to the sheet
for row_num, (value1, value2) in enumerate(zip(race_data, ethnicity_data), start=starting_row):
    sheet.cell(row=row_num, column=22, value=value1)
    sheet.cell(row=row_num, column=23, value=value2)

workbook.save(path)

OSError: [Errno 30] Read-only file system: '/media/cdac-c-15/Thunderpack/Dropbox/cardiac arrest project/cardiac_filtered_orginal .xlsx'